In [14]:
import numpy as np
import pandas as pd

from py.consts import DATA_DIR, DAYS_OF_WEEK, MONTHS_OF_YEAR
from py.utils import one_hot_encode

In [15]:
df = pd.read_pickle(DATA_DIR / "flitsers.pkl")

sample_size: int = 1000
print(f"number of records: {len(df)}")
df = df.sample(sample_size)
print(f"sampled to {sample_size} records")
    
df.head()

number of records: 96757
sampled to 1000 records


,id,datum,soort_weg,wegnummer,zijde,hm_paal,type_controle,tijd_van_melden,details,laatste_activiteit,...,weer_temp_min,weer_luchtdruk_hpa,weer_luchtvochtigheid_procent,weer_windsnelheid_m_per_sec,weer_windrichting_graden,weer_bewolking_procent,weer_regen_mm,weer_sneeuw_mm,weer_zonnestand,weer_locatie_naam
39790,59549,2020-04-20,nweg,N240,beide,5.7,Radar,14:44:37,\n</td,19:50:32.549953,...,16.00,1021.0,36.0,10.30,80.0,0.0,NaN,NaN,na_zonsopgang,Gemeente Wieringermeer
91125,5911,2016-09-24,snelweg,A28,rechts,196.3,Radar,07:40:33,\r\n,08:15:06.410072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
14107,93235,2022-02-22,nweg,N196,beide,0,Radar,11:09:29,on,14:31:12.225653,...,6.66,1013.0,84.0,8.94,210.0,75.0,NaN,NaN,na_zonsopgang,Schiphol
46833,50635,2019-09-25,nweg,N278,beide,9.4,Laser,14:00:08,bij ingber\n</td,16:50:36.269562,...,16.00,1006.0,67.0,8.20,230.0,90.0,NaN,NaN,na_zonsopgang,Margraten
25402,79527,2021-04-14,nweg,N257,beide,7.5,Radar,06:37:11,\n</td,12:50:27.937181,...,1.00,1030.0,80.0,1.54,160.0,90.0,NaN,NaN,voor_zonsopgang,Nieuw-Vossemeer


In [16]:
df.shape

(1000, 27)

In [17]:
dataset = df[["datum", "wegnummer", "zijde"]]

dataset["datum"] = pd.to_datetime(dataset.datum)
dataset["month"] = dataset.datum.dt.month_name()
dataset["day"] = dataset.datum.dt.day_name()
dataset = dataset.drop(columns=["datum"])

dataset["y"] = 1

dataset.head()

/tmp/ipykernel_1478/3700819593.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["datum"] = pd.to_datetime(dataset.datum)
/tmp/ipykernel_1478/3700819593.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["month"] = dataset.datum.dt.month_name()
/tmp/ipykernel_1478/3700819593.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,wegnummer,zijde,month,day,y
39790,N240,beide,April,Monday,1
91125,A28,rechts,September,Saturday,1
14107,N196,beide,February,Tuesday,1
46833,N278,beide,September,Wednesday,1
25402,N257,beide,April,Wednesday,1


In [18]:
# zijde
dataset = pd.concat([dataset, dataset.loc[(dataset.y == 1) & (dataset.zijde == "links")].assign(zijde="rechts", y=0)])
dataset = pd.concat([dataset, dataset.loc[(dataset.y == 1) & (dataset.zijde == "rechts")].assign(zijde="links", y=0)])

df.head()

,id,datum,soort_weg,wegnummer,zijde,hm_paal,type_controle,tijd_van_melden,details,laatste_activiteit,...,weer_temp_min,weer_luchtdruk_hpa,weer_luchtvochtigheid_procent,weer_windsnelheid_m_per_sec,weer_windrichting_graden,weer_bewolking_procent,weer_regen_mm,weer_sneeuw_mm,weer_zonnestand,weer_locatie_naam
39790,59549,2020-04-20,nweg,N240,beide,5.7,Radar,14:44:37,\n</td,19:50:32.549953,...,16.00,1021.0,36.0,10.30,80.0,0.0,NaN,NaN,na_zonsopgang,Gemeente Wieringermeer
91125,5911,2016-09-24,snelweg,A28,rechts,196.3,Radar,07:40:33,\r\n,08:15:06.410072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
14107,93235,2022-02-22,nweg,N196,beide,0,Radar,11:09:29,on,14:31:12.225653,...,6.66,1013.0,84.0,8.94,210.0,75.0,NaN,NaN,na_zonsopgang,Schiphol
46833,50635,2019-09-25,nweg,N278,beide,9.4,Laser,14:00:08,bij ingber\n</td,16:50:36.269562,...,16.00,1006.0,67.0,8.20,230.0,90.0,NaN,NaN,na_zonsopgang,Margraten
25402,79527,2021-04-14,nweg,N257,beide,7.5,Radar,06:37:11,\n</td,12:50:27.937181,...,1.00,1030.0,80.0,1.54,160.0,90.0,NaN,NaN,voor_zonsopgang,Nieuw-Vossemeer


In [19]:
negative_records_added = 0

for i, month in enumerate(MONTHS_OF_YEAR):
    remaining_months = MONTHS_OF_YEAR.copy()
    del remaining_months[i]

    records_on_this_month = dataset.loc[(dataset.y == 1) & (dataset.month == month)]
    records_on_this_month_count = len(records_on_this_month)

    print(f"{records_on_this_month_count} occurrences on {month}: adding {records_on_this_month_count * 6} negative records...")
    
    negative_records = []
    for remaining_month in remaining_months:
        negative_records.append(records_on_this_month.assign(month=remaining_month, y=0))
        
    dataset = pd.concat([dataset, *negative_records])

    negative_records_added += records_on_this_month_count * 6

print(f"added {negative_records_added} negative records")

98 occurrences on January: adding 588 negative records...
97 occurrences on February: adding 582 negative records...
99 occurrences on March: adding 594 negative records...
107 occurrences on April: adding 642 negative records...
77 occurrences on May: adding 462 negative records...
78 occurrences on June: adding 468 negative records...
93 occurrences on July: adding 558 negative records...
68 occurrences on August: adding 408 negative records...
84 occurrences on September: adding 504 negative records...
62 occurrences on October: adding 372 negative records...
74 occurrences on November: adding 444 negative records...
63 occurrences on December: adding 378 negative records...
added 6000 negative records


In [20]:
negative_records_added = 0

for i, day in enumerate(DAYS_OF_WEEK):
    remaining_days = DAYS_OF_WEEK.copy()
    del remaining_days[i]
    
    records_on_this_day = dataset.loc[(dataset.y == 1) & (dataset.day == day)]
    records_on_this_day_count = len(records_on_this_day)
    
    print(f"{records_on_this_day_count} occurrences on {day}: adding {records_on_this_day_count * 6} negative records...")
    
    negative_records = []
    for remaining_day in remaining_days:
        negative_records.append(records_on_this_day.assign(day=remaining_day, y=0))
        
    dataset = pd.concat([dataset, *negative_records])
    
    negative_records_added += records_on_this_day_count * 6

print(f"added {negative_records_added} negative records")

159 occurrences on Monday: adding 954 negative records...
194 occurrences on Tuesday: adding 1164 negative records...
188 occurrences on Wednesday: adding 1128 negative records...
154 occurrences on Thursday: adding 924 negative records...
139 occurrences on Friday: adding 834 negative records...
99 occurrences on Saturday: adding 594 negative records...
67 occurrences on Sunday: adding 402 negative records...
added 6000 negative records


In [21]:
road_numbers = dataset.wegnummer.unique()
negative_records_added = 0

for i, road_number in enumerate(road_numbers):
    remaining_roads = np.delete(road_numbers.copy(), [i])
    remaining_roads_count = len(remaining_roads)
    
    records_on_this_road = dataset.loc[(dataset.y == 1) & (dataset.wegnummer == road_number)]
    records_on_this_road_count = len(records_on_this_road)
    
    print(f"{records_on_this_road_count} occurrences on road {road_number}: adding {records_on_this_road_count * remaining_roads_count} negative records...")
    
    negative_records = []
    for remaining_road in remaining_roads:
        negative_records.append(records_on_this_road.assign(wegnummer=remaining_road, y=0))
        
    dataset = pd.concat([dataset, *negative_records])
        
    negative_records_added += records_on_this_road_count * remaining_roads_count

print(f"added {negative_records_added} negative records")

4 occurrences on road N240: adding 1072 negative records...
19 occurrences on road A28: adding 5092 negative records...
3 occurrences on road N196: adding 804 negative records...
2 occurrences on road N278: adding 536 negative records...
3 occurrences on road N257: adding 804 negative records...
1 occurrences on road N248: adding 268 negative records...
1 occurrences on road N379: adding 268 negative records...
8 occurrences on road N33: adding 2144 negative records...
1 occurrences on road N332: adding 268 negative records...
8 occurrences on road N305: adding 2144 negative records...
14 occurrences on road A6: adding 3752 negative records...
42 occurrences on road A2: adding 11256 negative records...
2 occurrences on road N48: adding 536 negative records...
16 occurrences on road N201: adding 4288 negative records...
3 occurrences on road N615: adding 804 negative records...
4 occurrences on road N351: adding 1072 negative records...
5 occurrences on road N702: adding 1340 negative r

In [23]:
print(dataset.shape)
dataset.head()

(286419, 5)


,wegnummer,zijde,month,day,y
39790,N240,beide,April,Monday,1
91125,A28,rechts,September,Saturday,1
14107,N196,beide,February,Tuesday,1
46833,N278,beide,September,Wednesday,1
25402,N257,beide,April,Wednesday,1


In [24]:
dataset.to_parquet(DATA_DIR / f"prepared_{sample_size}.parquet")

One-hot encode columns

In [ ]:
dataset = one_hot_encode(df=dataset, column="zijde")
dataset = one_hot_encode(df=dataset, column="day")
dataset = one_hot_encode(df=dataset, column="month")
dataset = one_hot_encode(df=dataset, column="wegnummer")
# dataset = one_hot_encode(df=dataset, column="type_controle")
# dataset = one_hot_encode(df=dataset, column="hm_paal")

In [181]:
dataset.head()
dataset.shape

(500, 412)

In [200]:
dataset.loc[(dataset.y == 1) & (dataset.day_Monday == True)]

,y,zijde_beide,zijde_links,zijde_rechts,type_controle_Laser,type_controle_Radar,day_Friday,day_Monday,day_Saturday,day_Sunday,...,hm_paal_9.6,hm_paal_9.7,hm_paal_9.8,hm_paal_90.2,hm_paal_90.7,hm_paal_91.5,hm_paal_93.7,hm_paal_95.6,hm_paal_99.0,hm_paal_99.2
0,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,False,True,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
389,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
390,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
391,1,True,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
